# 🚀 บทที่ 2.6: Optimizers Comparison - การเปรียบเทียบ Optimizers

## 🏃 เรื่องราวของนักวิ่ง 4 คน

สมมติคุณต้องวิ่งลงเขาให้ถึง**จุดต่ำสุด**เร็วที่สุด (Minimize Loss)

มีนักวิ่ง 4 คน วิ่งด้วยสไตล์ต่างกัน:

### 🐌 SGD (Stochastic Gradient Descent)
- วิ่ง**ช้าๆ แต่มั่นคง**
- มองพื้นรอบตัว แล้ววิ่งลงเนิน
- ติดหลุม (local minimum) ได้ง่าย
- **ข้อดี:** เรียบง่าย, เชื่อถือได้
- **ข้อเสีย:** ช้า, ติดหลุม

### 💨 Momentum
- วิ่ง**เร็วขึ้นเรื่อยๆ** เหมือนลูกบอลกลิ้ง
- ถ้าวิ่งไปทางเดียวกันนาน → เร่งความเร็ว
- ข้ามหลุมเล็กๆ ได้ (มี momentum)
- **ข้อดี:** เร็วกว่า SGD, ข้ามหลุมได้
- **ข้อเสีย:** อาจแกว่งข้ามจุดต่ำสุด

### 🎯 RMSprop
- วิ่ง**ปรับความเร็วตามภูมิประเทศ**
- ทางชัน → วิ่งช้า (ระวัง)
- ทางราบ → วิ่งเร็ว
- **ข้อดี:** ปรับตัวได้ดี
- **ข้อเสีย:** ซับซ้อนกว่า

### 🏆 Adam (Adaptive Moment Estimation)
- **รวมจุดเด่นของทุกคน!**
- มี momentum + ปรับความเร็วอัตโนมัติ
- **All-rounder** - เหมาะกับทุกสถานการณ์
- **ข้อดี:** เร็ว, ปรับตัวได้ดี, ใช้ง่าย
- **ข้อเสีย:** ใช้ memory มากกว่า

---

**คำถาม:** ใครจะถึงจุดต่ำสุดเร็วที่สุด? 🤔

**คำตอบ:** มาทดลองกันเลย! 🚀

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from tensorflow.keras.datasets import mnist
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
print("✅ Libraries loaded!")

---

## 🎯 Part 1: ทำความรู้จัก Optimizers

### Optimizer ทำอะไร?
อัปเดต weights ของโมเดลเพื่อลด loss

### สูตรพื้นฐาน:
```
weight_new = weight_old - learning_rate × gradient
```

In [ ]:
# สร้าง Loss Landscape (ภูเขา)
def create_loss_landscape():
    x = np.linspace(-5, 5, 100)
    y = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x, y)
    
    # Loss function (มีหลายหลุม)
    Z = (X**2 + Y**2) / 20 + np.sin(X) * np.cos(Y) + 5
    
    return X, Y, Z

X, Y, Z = create_loss_landscape()

# Plot 3D
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 6))

# 3D view
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax1.set_xlabel('Weight 1')
ax1.set_ylabel('Weight 2')
ax1.set_zlabel('Loss')
ax1.set_title('Loss Landscape (3D View)', fontsize=13, fontweight='bold')

# Contour view
ax2 = fig.add_subplot(122)
contour = ax2.contour(X, Y, Z, levels=20, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.set_xlabel('Weight 1')
ax2.set_ylabel('Weight 2')
ax2.set_title('Loss Landscape (Contour View)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🗻 This is the 'mountain' our optimizers need to descend!")
print("   Goal: Reach the LOWEST point (minimum loss)")

---

## 🔬 Part 2: เปรียบเทียบ Optimizers แบบละเอียด

### รายละเอียดแต่ละตัว:

In [ ]:
import pandas as pd

# สร้างตารางเปรียบเทียบ
comparison = pd.DataFrame({
    'Optimizer': ['SGD', 'SGD + Momentum', 'RMSprop', 'Adam', 'AdaGrad'],
    'Learning Rate': ['ต้องปรับเอง', 'ต้องปรับเอง', 'ปรับอัตโนมัติ', 'ปรับอัตโนมัติ', 'ปรับอัตโนมัติ'],
    'Speed': ['⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐'],
    'Stability': ['⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐'],
    'Memory': ['น้อย', 'ปานกลาง', 'ปานกลาง', 'มาก', 'ปานกลาง'],
    'Best For': [
        'งานง่ายๆ, เข้าใจง่าย',
        'CNNs, ชอบวิ่งเร็ว',
        'RNNs, Time Series',
        'ทุกอย่าง! (All-rounder)',
        'Sparse data'
    ],
    'Default LR': ['0.01', '0.01', '0.001', '0.001', '0.01']
})

print("📊 Optimizers Comparison Table")
print("="*100)
print(comparison.to_string(index=False))
print("\n⭐ = Rating (1-5 stars)")

In [ ]:
# Mathematical Formulas
print("\n📐 Mathematical Formulas:\n")
print("="*70)

print("\n1️⃣ SGD (Stochastic Gradient Descent):")
print("""
   w = w - α × ∇L
   
   w = weight
   α = learning rate
   ∇L = gradient of loss
""")

print("\n2️⃣ SGD with Momentum:")
print("""
   v = β × v + ∇L
   w = w - α × v
   
   v = velocity (momentum)
   β = momentum coefficient (usually 0.9)
""")

print("\n3️⃣ RMSprop:")
print("""
   s = β × s + (1-β) × (∇L)²
   w = w - α × ∇L / √(s + ε)
   
   s = squared gradient average
   ε = small number (1e-8) to avoid division by zero
""")

print("\n4️⃣ Adam (Adaptive Moment Estimation):")
print("""
   m = β₁ × m + (1-β₁) × ∇L        # First moment (mean)
   v = β₂ × v + (1-β₂) × (∇L)²     # Second moment (variance)
   
   m̂ = m / (1 - β₁ᵗ)               # Bias correction
   v̂ = v / (1 - β₂ᵗ)
   
   w = w - α × m̂ / (√v̂ + ε)
   
   β₁ = 0.9 (default)
   β₂ = 0.999 (default)
""")

---

## 🏁 Part 3: ทดลองจริงกับ MNIST Dataset

มาเทรนโมเดลจริงๆ กับข้อมูล MNIST แล้วดูว่า optimizer ไหนเร็วกว่ากัน!

In [ ]:
# Load MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# ใช้แค่ subset เพื่อความเร็ว
x_train_small = x_train[:10000]
y_train_small = y_train[:10000]
x_test_small = x_test[:2000]
y_test_small = y_test[:2000]

print("✅ MNIST Data loaded")
print(f"   Training: {x_train_small.shape}")
print(f"   Test: {x_test_small.shape}")

In [ ]:
# สร้างโมเดลเดียวกันสำหรับทุก optimizer
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Optimizers ที่จะทดลอง
optimizers_to_test = {
    'SGD': SGD(learning_rate=0.01),
    'SGD+Momentum': SGD(learning_rate=0.01, momentum=0.9),
    'RMSprop': RMSprop(learning_rate=0.001),
    'Adam': Adam(learning_rate=0.001)
}

print("🔧 Models ready!")
print(f"   Testing {len(optimizers_to_test)} optimizers")

In [ ]:
# Train แต่ละโมเดล
histories = {}
epochs = 20

print("🏃 Training models with different optimizers...\n")
print("="*70)

for name, optimizer in optimizers_to_test.items():
    print(f"\n🔄 Training with {name}...")
    
    # สร้างโมเดลใหม่
    model = create_model()
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train
    history = model.fit(
        x_train_small, y_train_small,
        epochs=epochs,
        batch_size=32,
        validation_data=(x_test_small, y_test_small),
        verbose=0  # ไม่แสดง progress
    )
    
    histories[name] = history.history
    
    final_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    print(f"   ✓ Final Train Acc: {final_acc:.4f}")
    print(f"   ✓ Final Val Acc: {final_val_acc:.4f}")

print("\n✅ All models trained!")

In [ ]:
# Plot เปรียบเทียบ
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = {'SGD': 'red', 'SGD+Momentum': 'orange', 'RMSprop': 'green', 'Adam': 'blue'}

# Training Loss
for name, history in histories.items():
    axes[0, 0].plot(history['loss'], label=name, color=colors[name], linewidth=2)
axes[0, 0].set_title('Training Loss', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Validation Loss
for name, history in histories.items():
    axes[0, 1].plot(history['val_loss'], label=name, color=colors[name], linewidth=2)
axes[0, 1].set_title('Validation Loss', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Training Accuracy
for name, history in histories.items():
    axes[1, 0].plot(history['accuracy'], label=name, color=colors[name], linewidth=2)
axes[1, 0].set_title('Training Accuracy', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Validation Accuracy
for name, history in histories.items():
    axes[1, 1].plot(history['val_accuracy'], label=name, color=colors[name], linewidth=2)
axes[1, 1].set_title('Validation Accuracy', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# สรุปผลลัพธ์
print("\n📊 Final Results Summary:\n")
print("="*80)

results = []
for name, history in histories.items():
    results.append({
        'Optimizer': name,
        'Final Train Acc': f"{history['accuracy'][-1]:.4f}",
        'Final Val Acc': f"{history['val_accuracy'][-1]:.4f}",
        'Final Train Loss': f"{history['loss'][-1]:.4f}",
        'Final Val Loss': f"{history['val_loss'][-1]:.4f}",
        'Converged at Epoch': np.argmin(history['val_loss']) + 1
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# หาผู้ชนะ
best_optimizer = results_df.loc[results_df['Final Val Acc'].idxmax(), 'Optimizer']
print(f"\n🏆 Winner: {best_optimizer}")
print(f"   Best validation accuracy!")

---

## 🎯 Part 4: เมื่อไหร่ควรใช้ Optimizer ไหน?

### Decision Guide:

In [ ]:
print("🎯 When to Use Which Optimizer?\n")
print("="*70)

print("\n1️⃣ Adam - ใช้เป็นค่าเริ่มต้น (Default Choice) ✅")
print("""
   Use when:
   • เริ่มโปรเจกต์ใหม่
   • ไม่แน่ใจว่าควรใช้อันไหน
   • Deep Learning ทั่วไป
   • ต้องการ converge เร็ว
   
   Settings:
   • learning_rate=0.001 (default)
   • beta_1=0.9, beta_2=0.999
""")

print("\n2️⃣ SGD + Momentum - สำหรับ CNNs ที่ต้องการ generalize ดี")
print("""
   Use when:
   • Image Classification
   • Computer Vision tasks
   • ต้องการโมเดลที่ generalize ดีกว่า (แม้ช้ากว่า)
   • มีเวลาเทรนนาน
   
   Settings:
   • learning_rate=0.01
   • momentum=0.9
   • ใช้ Learning Rate Scheduling
""")

print("\n3️⃣ RMSprop - สำหรับ RNNs และ Time Series")
print("""
   Use when:
   • Recurrent Neural Networks (RNNs)
   • LSTMs, GRUs
   • Time Series Prediction
   • ปัญหา Non-stationary
   
   Settings:
   • learning_rate=0.001
   • rho=0.9
""")

print("\n4️⃣ AdaGrad - สำหรับ Sparse Data")
print("""
   Use when:
   • Natural Language Processing
   • ข้อมูลที่ sparse (เต็มไปด้วย 0)
   • Word embeddings
   
   Settings:
   • learning_rate=0.01
""")

print("\n5️⃣ Plain SGD - สำหรับเรียนรู้พื้นฐาน")
print("""
   Use when:
   • เรียนรู้ทฤษฎี
   • โจทย์ง่ายๆ
   • ต้องการเข้าใจ optimization
   
   ❌ ไม่แนะนำสำหรับ production
""")

---

## 💡 Part 5: Tips & Tricks

### Learning Rate Scheduling

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler

print("📈 Learning Rate Scheduling Strategies:\n")
print("="*70)

print("\n1️⃣ ReduceLROnPlateau (แนะนำ!)")
print("""
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,           # ลด LR เหลือ 50%
    patience=5,           # รอ 5 epochs
    min_lr=1e-7
)

model.fit(..., callbacks=[reduce_lr])

→ ถ้า val_loss ไม่ดีขึ้น 5 epochs → ลด LR
""")

print("\n2️⃣ Step Decay")
print("""
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 10
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr

lr_scheduler = LearningRateScheduler(step_decay)

→ ลด LR ทุกๆ 10 epochs
""")

print("\n3️⃣ Cosine Annealing")
print("""
from tensorflow.keras.optimizers.schedules import CosineDecay

lr_schedule = CosineDecay(
    initial_learning_rate=0.01,
    decay_steps=1000
)

optimizer = Adam(learning_rate=lr_schedule)

→ LR ลดลงตาม cosine curve
""")

In [ ]:
# Visualize Learning Rate Schedules
epochs_range = np.arange(1, 51)

# Constant
lr_constant = np.ones(50) * 0.01

# Step Decay
lr_step = [0.01 * (0.5 ** (e // 10)) for e in epochs_range]

# Exponential Decay
lr_exp = [0.01 * (0.95 ** e) for e in epochs_range]

# Cosine
lr_cosine = [0.01 * (0.5 * (1 + np.cos(np.pi * e / 50))) for e in epochs_range]

# Plot
plt.figure(figsize=(12, 6))
plt.plot(epochs_range, lr_constant, 'b-', linewidth=2, label='Constant')
plt.plot(epochs_range, lr_step, 'r-', linewidth=2, label='Step Decay')
plt.plot(epochs_range, lr_exp, 'g-', linewidth=2, label='Exponential Decay')
plt.plot(epochs_range, lr_cosine, 'm-', linewidth=2, label='Cosine Annealing')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Learning Rate', fontsize=12)
plt.title('Learning Rate Scheduling Strategies', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print("💡 แนะนำ: ใช้ ReduceLROnPlateau เพราะปรับตาม performance จริง!")

---

## 📝 สรุปบทเรียน

### ✅ Key Takeaways:

#### 1. **Optimizer Ranking (โดยทั่วไป):**
```
🏆 Adam          → เร็ว, ใช้ง่าย, เหมาะกับเกือบทุกอย่าง
🥈 SGD+Momentum  → Generalize ดี, เหมาะกับ CV
🥉 RMSprop       → เหมาะกับ RNN
```

#### 2. **แนะนำสำหรับมือใหม่:**
- เริ่มด้วย **Adam** เสมอ
- Learning rate = 0.001
- ใช้ ReduceLROnPlateau

#### 3. **แนะนำสำหรับมือโปร:**
- CNNs: SGD + Momentum + LR Scheduling
- RNNs: RMSprop หรือ Adam
- NLP: Adam

#### 4. **Common Mistakes:**
- ❌ Learning rate สูงเกิน → ไม่ converge
- ❌ Learning rate ต่ำเกิน → ช้ามาก
- ❌ ไม่ใช้ LR scheduling → stuck ที่ plateau
- ❌ เปลี่ยน optimizer กลางทาง → weights เพี้ยน

#### 5. **Best Practices:**
- ✅ เริ่มด้วย Adam
- ✅ ใช้ Learning Rate Finder (optional)
- ✅ Monitor training curves
- ✅ ใช้ ReduceLROnPlateau
- ✅ Save best model

### 🎓 ยินดีด้วย!

คุณได้เรียนรู้ครบ **บทที่ 2: Data Training** แล้ว! 🎉

ตอนนี้คุณรู้:
- โหลดข้อมูลจาก CSV, Images, APIs ✅
- ทำความสะอาดข้อมูล (Missing, Outliers, Inconsistencies) ✅
- Transform ข้อมูล (Normalization, Standardization) ✅
- Data Augmentation ✅
- แบ่ง Train/Test + หลีกเลี่ยง Data Leakage ✅
- เลือก Optimizer ที่เหมาะสม ✅

### 🚀 Next Chapter:

👉 [บทที่ 3: CNN และการประมวลผลภาพ](../../03_CNN_Image_Processing/README.md)

พร้อมสร้าง Image Classifier แล้วหรือยัง? 📸